In [1]:
from mlptools.io.read import read_from_format
from mlptools.io.write import write_from_atoms
from mlptools.utils.constants import get_all_si_atoms
import os
import pickle
import shutil

/Users/y1u0d2/opt/anaconda3/envs/mlp/lib/python3.11/site-packages/ovito/_extensions/averaging.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? This will likely lead to conflicts with existing libraries in the Anaconda environment, and import of the OVITO module may fail with an error related to the Qt framework. To fix this, please uninstall the ovito pip package by running 'pip uninstall -y ovito PySide6' and then install the OVITO Anaconda package provided by OVITO GmbH. Visit https://docs.ovito.org/python/introduction/installation.html for further instructions. If you would rather like to ignore this warning message, add the following code to the top of your Python script:

  import warnings
  warnings.filterwarnings('ignore', message='.*OVITO.*PyPI')

  import ovito._extensions.pyscript


In [2]:
lower_limit = 1.6
upper_limit = 4.0
all_atoms = get_all_si_atoms(device='local')
print('number of atoms', len(all_atoms))

all_atoms_filtered = []
for i, atoms in enumerate(all_atoms):
    if i % 100 == 0:
        print(f'{i} / {len(all_atoms)}')

    if lower_limit <= atoms.distance_btw_nearest_neighbor <= upper_limit:
        all_atoms_filtered.append(atoms)

print('number of atoms after filtering', len(all_atoms_filtered))

number of atoms 82574
0 / 82574
100 / 82574
200 / 82574
300 / 82574
400 / 82574
500 / 82574
600 / 82574
700 / 82574
800 / 82574
900 / 82574
1000 / 82574
1100 / 82574
1200 / 82574
1300 / 82574
1400 / 82574
1500 / 82574
1600 / 82574
1700 / 82574
1800 / 82574
1900 / 82574
2000 / 82574
2100 / 82574
2200 / 82574
2300 / 82574
2400 / 82574
2500 / 82574
2600 / 82574
2700 / 82574
2800 / 82574
2900 / 82574
3000 / 82574
3100 / 82574
3200 / 82574
3300 / 82574
3400 / 82574
3500 / 82574
3600 / 82574
3700 / 82574
3800 / 82574
3900 / 82574
4000 / 82574
4100 / 82574
4200 / 82574
4300 / 82574
4400 / 82574
4500 / 82574
4600 / 82574
4700 / 82574
4800 / 82574
4900 / 82574
5000 / 82574
5100 / 82574
5200 / 82574
5300 / 82574
5400 / 82574
5500 / 82574
5600 / 82574
5700 / 82574
5800 / 82574
5900 / 82574
6000 / 82574
6100 / 82574
6200 / 82574
6300 / 82574
6400 / 82574
6500 / 82574
6600 / 82574
6700 / 82574
6800 / 82574
6900 / 82574
7000 / 82574
7100 / 82574
7200 / 82574
7300 / 82574
7400 / 82574
7500 / 82574
76

In [3]:
import random

n_sample = 100
idx_mp_three = [idx for idx, atoms in enumerate(all_atoms_filtered) if atoms.structure_id == 'mp-three']
sampled_idx_mp_three = random.sample(idx_mp_three, 100)
remove_idx_mp_three = set(idx_mp_three) - set(sampled_idx_mp_three)

print(f'Original: {len(idx_mp_three)}, Sampled: {len(sampled_idx_mp_three)}')

Original: 816, Sampled: 100


In [4]:
all_atoms_sampled = []
for idx, atoms in enumerate(all_atoms_filtered):
    if idx in remove_idx_mp_three:
        continue
    all_atoms_sampled.append(atoms)

In [5]:
import collections

structure_id_list = [atoms.structure_id for atoms in all_atoms_sampled]
c = collections.Counter(structure_id_list)

print(c)
print(f'Total number of data: {len(all_atoms_sampled)}')

Counter({'mp-149': 39320, 'mp-92': 5039, 'mp-1079649': 3653, 'mp-165': 3422, 'mp-571520': 3142, 'mp-slab': 261, 'mp-three': 100, 'mp-four': 62, 'mp-149_dimer': 36})
Total number of data: 55035


In [6]:
path2out = '/Users/y1u0d2/desktop/Lab/result/n2p2/Si/96/data'
# all_dirs = list(filter(lambda x: 'mp-149_dimer' not in x, all_dirs))
with open(f'{path2out}/input.data', mode='a') as f:
    for atoms in all_atoms_sampled:
        try:
            f.write('\n'.join(write_from_atoms(atoms, format='n2p2')))
        except:
            print(atoms.structure_id)
            continue

# For prediction

In [4]:
from glob import glob
from mlptools.atoms.atom import MLPAtoms
import numpy as np

path2atoms = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection'
all_atoms = glob(f'{path2atoms}/*.pkl')
print('number of atoms', len(all_atoms))

def ase_atom_to_mlpatom(path2atom, ase_atom):
    n_atoms = ase_atom.get_global_number_of_atoms()
    cell = np.array([vec for vec in ase_atom.cell])
    coord = ase_atom.get_positions()
    force = np.zeros((n_atoms, 3))
    energy = 0.0
    total_magnetization = 0.0
    structure_id = 'substrate'
    symbols = f'Si{n_atoms}'
    return MLPAtoms(
        cell=cell,
        coord=coord,
        force=force,
        energy=energy,
        n_atoms=n_atoms,
        total_magnetization=total_magnetization,
        structure_id=structure_id,
        symbols=symbols,
        path=path2atom
    )

number of atoms 100


In [3]:
path2out = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/n2p2_input'
path2model = '/Users/y1u0d2/desktop/Lab/result/lammps/models/Si/18'
all_mlpatoms = []
for atoms in all_atoms:
    #load atoms from pkl
    with open(atoms, mode='rb') as f:
        ase_atom = pickle.load(f)

    ase_atom.set_chemical_symbols(f'Si{ase_atom.get_global_number_of_atoms()}')
    mlpatom = ase_atom_to_mlpatom(path2atom=atoms, ase_atom=ase_atom)

    frame_id = mlpatom.path.split('/')[-1].split('.')[0]
    path2target = os.path.join(path2out, f'{frame_id}')
    os.makedirs(path2target, exist_ok=True)
    # copy input.nn scaling.data weights.014.data from path2model to path2target
    shutil.copy(os.path.join(path2model, 'input.nn'), path2target)
    shutil.copy(os.path.join(path2model, 'scaling.data'), path2target)
    shutil.copy(os.path.join(path2model, 'weights.014.data'), path2target)
    # write input.data
    with open(os.path.join(path2target, 'input.data'), mode='w') as f:
        f.write('\n'.join(write_from_atoms(mlpatom, format='n2p2')))

number of mlpatoms 0


## LAMMPSのinputファイルを作成する

In [80]:
path2template = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/lmp_input/template'
path2out = '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/atoms_near_injection/lmp_input'

def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list


all_mlpatoms = []
for atoms in all_atoms:
    #load atoms from pkl
    with open(atoms, mode='rb') as f:
        ase_atom = pickle.load(f)

    ase_atom.set_chemical_symbols(f'Si{ase_atom.get_global_number_of_atoms()}')
    mlpatom = ase_atom_to_mlpatom(path2atom=atoms, ase_atom=ase_atom)

    with open(os.path.join(path2template, 'lmp.in'), mode='r') as f:
        lines = [s.strip() for s in f.readlines()]

    scaled_positions = ase_atom.get_scaled_positions()
    cell = np.array([vec for vec in ase_atom.cell])

    cell_for_lmp = []
    for i, vec in enumerate(cell):
        vec = list(map(lambda x: str(x), vec))
        cell_for_lmp.append([f'a{i+1}'] + vec + ['&'])

    scaled_positions_for_lmp = []
    for scaled_position in scaled_positions:
        scaled_position_str = []
        for x in scaled_position:
            if x > 1.0:
                x = 0.999999
            scaled_position_str.append(str('{:.6f}'.format(x)))  
        # scaled_position = list(map(lambda x: str('{:.6f}'.format(x)) , scaled_position))
        scaled_positions_for_lmp.append(['basis'] + scaled_position_str + ['&'])

    cell_for_lmp = [' '.join(cell) for cell in cell_for_lmp]
    scaled_positions_for_lmp = [' '.join(scaled_position) for scaled_position in scaled_positions_for_lmp]
    for i, l in enumerate(lines):
        if 'a1 ' in l:
            lines[i:i+3] = cell_for_lmp
            lines[i+3] = scaled_positions_for_lmp

    lines = flatten_list(lines)

    frame_id = mlpatom.path.split('/')[-1].split('.')[0]
    path2target = os.path.join(path2out, f'{frame_id}')
    if os.path.exists(path2target):
        raise Exception(f'{path2target} already exists')
    else:
        os.makedirs(path2target)
    
    with open(os.path.join(path2target, 'lmp.in'), mode='w') as f:
        # write lines
        f.write('\n'.join(lines))

In [74]:
cell_for_lmp

['a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &']

In [76]:
lines

['atom_style      atomic',
 'units           metal',
 'boundary        p p p',
 'atom_modify map array sort 0 0',
 '',
 'lattice custom 1.0 &',
 'a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &',
 ['basis 0.000133 0.000133 0.000035 &',
  'basis 0.000133 0.066800 0.017732 &',
  'basis 0.066800 0.000133 0.017732 &',
  'basis 0.066800 0.066800 0.000035 &',
  'basis 0.033467 0.033467 0.008884 &',
  'basis 0.033467 0.100133 0.026580 &',
  'basis 0.100133 0.033467 0.026580 &',
  'basis 0.100133 0.100133 0.008884 &',
  'basis 0.133467 0.000133 0.000035 &',
  'basis 0.133467 0.066800 0.017732 &',
  'basis 0.200133 0.000133 0.017732 &',
  'basis 0.200133 0.066800 0.000035 &',
  'basis 0.166800 0.033467 0.008884 &',
  'basis 0.166800 0.100133 0.026580 &',
  'basis 0.233467 0.033467 0.026580 &',
  'basis 0.233467 0.100133 0.008884 &',
  'basis 0.000133 0.133467 0.000035 &',
  'basis 0.000133 0.200133 0.017732 &',
  'basis 0.066800 0.133467 0

In [71]:
def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

['atom_style      atomic',
 'units           metal',
 'boundary        p p p',
 'atom_modify map array sort 0 0',
 '',
 'lattice custom 1.0 &',
 'a1 40.724999999999994 0.0 0.0 &',
 'a2 0.0 40.724999999999994 0.0 &',
 'a3 0.0 0.0 153.41921902474164 &',
 'basis 0.000133 0.000133 0.000035 &',
 'basis 0.000133 0.066800 0.017732 &',
 'basis 0.066800 0.000133 0.017732 &',
 'basis 0.066800 0.066800 0.000035 &',
 'basis 0.033467 0.033467 0.008884 &',
 'basis 0.033467 0.100133 0.026580 &',
 'basis 0.100133 0.033467 0.026580 &',
 'basis 0.100133 0.100133 0.008884 &',
 'basis 0.133467 0.000133 0.000035 &',
 'basis 0.133467 0.066800 0.017732 &',
 'basis 0.200133 0.000133 0.017732 &',
 'basis 0.200133 0.066800 0.000035 &',
 'basis 0.166800 0.033467 0.008884 &',
 'basis 0.166800 0.100133 0.026580 &',
 'basis 0.233467 0.033467 0.026580 &',
 'basis 0.233467 0.100133 0.008884 &',
 'basis 0.000133 0.133467 0.000035 &',
 'basis 0.000133 0.200133 0.017732 &',
 'basis 0.066800 0.133467 0.017732 &',
 'basis